# An attempt to understand missing values

People keep on asking what to do with missing values. It depends on the situation, but I am certain 9/10 times you do not want to do anything and you would actually treat `NA` as a number. Well this itself is a hard problem - where do I put `NA` in a number space? There are many things to do, but what really matters is to understand how `NA` are generated in the first place. Having that knowledge you can find unexpected solutions!

Let's dive in into AMEX competition dataset. We are going to use the curated [dataset](https://www.kaggle.com/datasets/raddar/amex-data-integer-dtypes-parquet-format) which contains discovered integer column types.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)

train = pd.read_parquet('../input/amex-data-integer-dtypes-parquet-format/train.parquet')
train[train==-1] = np.nan # revert -1 to na encoding as in the original dataset

Let's calculate `NA` counts by feature. The idea is to identify feature clusters having the same `NA` values for each row - that is what I will be focusing in this notebook onwards. Let's explore clusters having more than 10 features.

In [2]:
cols = sorted(train.columns[2:].tolist())
nas = train[cols].isna().sum(axis=0).reset_index(name='NA_count')
nas['group_count'] = nas.loc[nas.NA_count>0].groupby('NA_count').transform('count')
clusters = nas.loc[nas.group_count>10].sort_values(['NA_count','index']).groupby('NA_count')['index'].apply(list).values
clusters

array([list(['B_16', 'B_19', 'B_2', 'B_20', 'B_22', 'B_26', 'B_27', 'B_3', 'B_30', 'B_33', 'B_38', 'D_41', 'D_54']),
       list(['D_103', 'D_104', 'D_107', 'D_128', 'D_129', 'D_130', 'D_131', 'D_139', 'D_141', 'D_143', 'D_145']),
       list(['D_113', 'D_114', 'D_115', 'D_116', 'D_117', 'D_118', 'D_119', 'D_120', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125'])],
      dtype=object)

There are 3 large clusters having many overlapping `NA` rows! Let's analyse each cluster!

In [3]:
cluster0_customers = set(train.loc[pd.isnull(train[clusters[0][0]]),'customer_ID'])
cluster1_customers = set(train.loc[pd.isnull(train[clusters[1][0]]),'customer_ID'])
cluster2_customers = set(train.loc[pd.isnull(train[clusters[2][0]]),'customer_ID'])

## cluster ZERO

Let's just simply look at how the customer profile looks for this feature cluster:

In [4]:
train.loc[train.customer_ID.isin(cluster0_customers),['customer_ID','S_2']+clusters[0]].head(100)

,customer_ID,S_2,B_16,B_19,B_2,B_20,B_22,B_26,B_27,B_3,B_30,B_33,B_38,D_41,D_54
12883,009b4f146ac20c9e528e23137b3fbef84856f327124ade...,2017-11-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12884,009b4f146ac20c9e528e23137b3fbef84856f327124ade...,2017-12-31,0.0,0.0,0.818834,0.0,0.0,0.001448,0.008550,0.000271,0.0,1.0,1.0,0.0,1.0
12885,009b4f146ac20c9e528e23137b3fbef84856f327124ade...,2018-01-05,1.0,0.0,0.074005,1.0,0.0,0.003338,0.007771,0.033593,0.0,0.0,4.0,0.0,1.0
12886,009b4f146ac20c9e528e23137b3fbef84856f327124ade...,2018-02-15,2.0,50.0,0.000795,2.0,1.0,0.009936,0.002902,0.201688,2.0,0.0,4.0,0.0,1.0
12887,009b4f146ac20c9e528e23137b3fbef84856f327124ade...,2018-03-03,3.0,66.0,0.014707,3.0,1.0,0.005296,0.007223,0.281117,1.0,0.0,4.0,0.0,1.0
15183,00b686ade48123a567b066546d791c825ddd735b1bbd9c...,2017-03-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15184,00b686ade48123a567b066546d791c825ddd735b1bbd9c...,2017-04-29,0.0,0.0,0.812942,0.0,0.0,0.001270,0.007633,0.009499,0.0,1.0,1.0,0.0,1.0
15185,00b686ade48123a567b066546d791c825ddd735b1bbd9c...,2017-05-08,1.0,0.0,0.099998,1.0,0.0,0.001971,0.008844,0.026143,0.0,0.0,4.0,0.0,1.0
15186,00b686ade48123a567b066546d791c825ddd735b1bbd9c...,2017-06-07,2.0,0.0,0.108898,2.0,0.0,0.008232,0.008512,0.035714,0.0,0.0,4.0,0.0,1.0
15187,00b686ade48123a567b066546d791c825ddd735b1bbd9c...,2017-07-08,3.0,0.0,0.097360,3.0,0.0,0.008810,0.009887,0.044543,0.0,0.0,4.0,0.0,1.0


Were you able to notice the pattern? What if I told you that `NA` appears only on the first observation for each customer! So for this feature cluster `NA` represents fresh credit card accounts with probably zero balance!

Let's double check that `NA` only appears on the first row:

In [5]:
# first row
print('first row of the customer')
print(train.loc[train.customer_ID.isin(cluster0_customers)].groupby('customer_ID')[clusters[0]].head(1).isna().sum(axis=0))

# any row
print('any row of the customer')
print(train[clusters[0]].isna().sum(axis=0))

first row of the customer
B_16    2016
B_19    2016
B_2     2016
B_20    2016
B_22    2016
B_26    2016
B_27    2016
B_3     2016
B_30    2016
B_33    2016
B_38    2016
D_41    2016
D_54    2016
dtype: int64
any row of the customer
B_16    2016
B_19    2016
B_2     2016
B_20    2016
B_22    2016
B_26    2016
B_27    2016
B_3     2016
B_30    2016
B_33    2016
B_38    2016
D_41    2016
D_54    2016
dtype: int64


Assumption correct ^

### How could I use this information?

We know that the dataset has varying number of available observations for each customer (N=1,...,13). It is so far obvious that `last` type features are the strongest ones to build a model.

For N=1 it gets a bit interesting, because this `NA` propagates to `last` type features. This may cause headache for non-GBDT models. 

Let me show you what I mean - the distribution of `B_33_last` as an example:

In [6]:
train.groupby('customer_ID').tail(1).groupby('B_33', dropna=False)['customer_ID'].count()

B_33
0.0    194132
1.0    264750
NaN        31
Name: customer_ID, dtype: int64

There are 31 `NA` values, each coming from customers having a single entry in the data. So if you had the feature like `number_of_observations`, you could add dummy value for the counter meaning `1 observation and B_X is NA` - I will encode this as 0.5 in the example below:

In [7]:
train['number_of_observations'] = train.groupby('customer_ID')['customer_ID'].transform('count')
print(train.groupby('customer_ID').tail(1).groupby('number_of_observations')['customer_ID'].count().reset_index())
train.loc[pd.isnull(train.B_33) & (train.number_of_observations==1),'number_of_observations'] = 0.5
print(train.groupby('customer_ID').tail(1).groupby('number_of_observations')['customer_ID'].count().reset_index())

    number_of_observations  customer_ID
0                        1         5120
1                        2         6098
2                        3         5778
3                        4         4673
4                        5         4671
5                        6         5515
6                        7         5198
7                        8         6110
8                        9         6411
9                       10         6721
10                      11         5961
11                      12        10623
12                      13       386034
    number_of_observations  customer_ID
0                      0.5           31
1                      1.0         5089
2                      2.0         6098
3                      3.0         5778
4                      4.0         4673
5                      5.0         4671
6                      6.0         5515
7                      7.0         5198
8                      8.0         6110
9                      9.0         6411


And now you are safe to set all `last` type features' `NA` to zero without losing any information. This means less categories for tree methods, and less hassle for building your neural networks.

In [8]:
train.loc[train.number_of_observations==0.5,['customer_ID','S_2','number_of_observations']+clusters[0]].fillna(0)

,customer_ID,S_2,number_of_observations,B_16,B_19,B_2,B_20,B_22,B_26,B_27,B_3,B_30,B_33,B_38,D_41,D_54
89890,042a81bae597726cf7122265ab1533e99866af050f9e54...,2018-03-04,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
318316,0ee5a69758e52a8ee2d540da11e433b7d007a165f85ae6...,2018-03-08,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
438247,1465704aae5ab7eceff36d8f474785899d53db0f2a73f0...,2018-03-06,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
641155,1ddd5db94ed3e69d3f155ba0a08d3a7d9ae19832c9f63b...,2018-03-01,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
722891,2194bf8deb7ebaaff774e7c2b1255c69b871170aff3b63...,2018-03-20,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
840431,270048c153b02328383135b66e6f0ec336087a0f367348...,2018-03-08,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1047745,3065217f59c58c277ebe71dc712ec9d877878e73cdcbbc...,2018-03-02,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1164950,35ccbbca1e8f05a5c0668559af4981aed686efacb0d7ab...,2018-03-07,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1230569,38bae54ab18f744450e4cb309af2e26c1401d1c4f61a28...,2018-03-06,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1612641,4a4617a5d8082c2b119bd60cdd94af4d8f08b5d5c1ee15...,2018-03-07,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Another observation could be made that for aggregate type features (min/max/mean,...), it is safe to not include `NA` in any way (the information is already stored in `number_of_observations=0.5`). So using functions like `np.nanmin`, `np.nanmax`, etc. are prefered

## cluster ONE

Again, simply look at what the data has to show:

In [9]:
train.loc[train.customer_ID.isin(cluster1_customers),['customer_ID','S_2']+clusters[1]].head(100)

,customer_ID,S_2,D_103,D_104,D_107,D_128,D_129,D_130,D_131,D_139,D_141,D_143,D_145
133,00013c6e1cec7c21bede7cb319f1e28eb994f5625257f4...,2017-03-25,1.0,0.955805,1.0,1.006618,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
134,00013c6e1cec7c21bede7cb319f1e28eb994f5625257f4...,2017-04-14,1.0,0.961292,1.0,1.003446,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
135,00013c6e1cec7c21bede7cb319f1e28eb994f5625257f4...,2017-05-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,00013c6e1cec7c21bede7cb319f1e28eb994f5625257f4...,2017-06-12,1.0,0.965208,1.0,1.000898,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
137,00013c6e1cec7c21bede7cb319f1e28eb994f5625257f4...,2017-07-01,1.0,0.964032,1.0,1.005529,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
138,00013c6e1cec7c21bede7cb319f1e28eb994f5625257f4...,2017-08-18,1.0,0.963906,1.0,1.000798,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
139,00013c6e1cec7c21bede7cb319f1e28eb994f5625257f4...,2017-09-09,1.0,0.961740,1.0,1.000112,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
140,00013c6e1cec7c21bede7cb319f1e28eb994f5625257f4...,2017-10-02,1.0,0.966617,1.0,1.002967,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
141,00013c6e1cec7c21bede7cb319f1e28eb994f5625257f4...,2017-11-13,1.0,0.965321,1.0,0.999608,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
142,00013c6e1cec7c21bede7cb319f1e28eb994f5625257f4...,2017-12-21,1.0,0.967010,1.0,1.003577,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.0


This appears similar to cluster ZERO. However, not all `NA` are in the first row!

In [10]:
# first row
print('first row of the customer')
print(train.loc[train.customer_ID.isin(cluster1_customers)].groupby('customer_ID')[clusters[1]].head(1).isna().sum(axis=0))

# any row
print('any row of the customer')
print(train[clusters[1]].isna().sum(axis=0))

first row of the customer
D_103    49166
D_104    49166
D_107    49166
D_128    49166
D_129    49166
D_130    49166
D_131    49166
D_139    49166
D_141    49166
D_143    49166
D_145    49166
dtype: int64
any row of the customer
D_103    101548
D_104    101548
D_107    101548
D_128    101548
D_129    101548
D_130    101548
D_131    101548
D_139    101548
D_141    101548
D_143    101548
D_145    101548
dtype: int64


In fact, half of the `NA` are in first row and other are in random positions in the timeline. I have not yet found any pattern for the latter.

So, what can be done!?

For first row `NA` we can apply the same logic as in cluster ZERO - will leave this as an exercise for you :)

As for randomly appearing `NA` I think it is worth doing some kind of imputation. Can't stress this enough, but THE MODEL MUST VERIFY THAT IMPUTATION IS A VALID APPROACH! It may be obvious for you that imputation is a legitimate strategy, but the model you will build has to confirm it, meaning that the model with imputed values should perform no worse than with originally missing values. This is how you test the hypothesis if the `NA` is missing at random or not.


What would I do here? I would probably take average values of `t-1` and `t+1` values for `NA` at timestep `t`. It is pretty straightforward to do so I won't cover this in this notebook. Let's better finish up with the last cluster!

## cluster TWO

Once again, simply look at what the data has to show:

In [11]:
train.loc[train.customer_ID.isin(cluster2_customers),['customer_ID','S_2']+clusters[2]].head(100)

,customer_ID,S_2,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125
78,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-04-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-05-12,2.0,0.0,0.008340,0.0,3.0,0.012875,0.006543,0.0,0.012269,1.0,0.0,2.0,1.0
81,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-06-10,2.0,0.0,0.008266,0.0,3.0,0.007987,0.012051,0.0,0.005338,1.0,1.0,2.0,1.0
82,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-07-19,2.0,0.0,0.009807,0.0,3.0,0.015161,0.006893,0.0,0.008296,1.0,1.0,2.0,1.0
83,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-08-09,2.0,0.0,0.011006,0.0,3.0,0.013832,0.006022,0.0,0.003994,1.0,1.0,2.0,1.0
84,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-09-06,2.0,0.0,0.014584,0.0,3.0,0.017930,0.011985,0.0,0.011438,1.0,1.0,2.0,1.0
85,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-10-15,2.0,0.0,0.011122,0.0,3.0,0.014859,0.017973,0.0,0.017412,1.0,1.0,2.0,1.0
86,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-11-10,2.0,0.0,0.014312,0.0,3.0,0.015011,0.010192,0.0,0.015564,1.0,1.0,2.0,1.0
87,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-12-19,2.0,0.0,0.022815,0.0,3.0,0.023212,0.019892,0.0,0.017386,1.0,0.0,2.0,0.0


If you looked closely enough you would identify that we have rolling `NA` values and the rolling stops when the first numeric information appears, and `NA` no longer reappears. Let's confirm this with a code:

In [12]:
train['cluster2_NA'] = 0
train.loc[pd.isnull(train[clusters[2][0]]),'cluster2_NA'] = 1
train['rank'] = train.groupby('customer_ID')['S_2'].rank()

not_na = train.loc[train.cluster2_NA==0].groupby('customer_ID')['rank'].agg(('min','max'))
yes_na = train.loc[train.cluster2_NA==1].groupby('customer_ID')['rank'].agg(('min','max'))
c2 = pd.merge(yes_na,not_na,on='customer_ID')

The idea is that max rank of `NA` should be lower than min rank of non-`NA`. Let's see if it holds:

In [13]:
c2.loc[c2.max_x>c2.min_y].shape[0], c2.loc[c2.max_x<c2.min_y].shape[0]

(6879, 60922)

In [14]:
#example customer timeframe showing that `NA` can start from the first observation
train.loc[train.customer_ID == 'fffe2bc02423407e33a607660caeed076d713d8a5ad32321530e92704835da88',['customer_ID','S_2']+clusters[2]]

,customer_ID,S_2,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125
5531263,fffe2bc02423407e33a607660caeed076d713d8a5ad323...,2017-08-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5531264,fffe2bc02423407e33a607660caeed076d713d8a5ad323...,2017-09-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5531265,fffe2bc02423407e33a607660caeed076d713d8a5ad323...,2017-10-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5531266,fffe2bc02423407e33a607660caeed076d713d8a5ad323...,2017-11-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5531267,fffe2bc02423407e33a607660caeed076d713d8a5ad323...,2017-12-14,2.0,0.0,0.038071,0.0,4.0,0.038798,0.039798,1.0,0.029313,1.0,0.0,2.0,0.0
5531268,fffe2bc02423407e33a607660caeed076d713d8a5ad323...,2018-01-18,2.0,0.0,0.044786,0.0,4.0,0.042089,0.039961,1.0,0.023081,1.0,0.0,2.0,0.0
5531269,fffe2bc02423407e33a607660caeed076d713d8a5ad323...,2018-02-23,2.0,0.0,0.042733,0.0,4.0,0.040142,0.041606,1.0,0.027084,1.0,0.0,2.0,0.0
5531270,fffe2bc02423407e33a607660caeed076d713d8a5ad323...,2018-03-17,2.0,0.0,0.046845,0.0,0.0,0.006365,0.005157,1.0,0.036950,1.0,0.0,2.0,0.0


In [15]:
#example customer timeframe showing that `NA` does not always start from the first observation
train.loc[train.customer_ID == '002259d195fcb87b15b98503ac5d2c4d29bc8383053ca3ad8951c598fbb6812a',['customer_ID','S_2']+clusters[2]]

,customer_ID,S_2,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125
2878,002259d195fcb87b15b98503ac5d2c4d29bc8383053ca3...,2017-05-24,2.0,0.0,0.048968,0.0,4.0,0.047960,0.047548,0.0,0.335114,2.0,1.0,6.0,0.0
2879,002259d195fcb87b15b98503ac5d2c4d29bc8383053ca3...,2017-06-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2880,002259d195fcb87b15b98503ac5d2c4d29bc8383053ca3...,2017-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2881,002259d195fcb87b15b98503ac5d2c4d29bc8383053ca3...,2017-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2882,002259d195fcb87b15b98503ac5d2c4d29bc8383053ca3...,2017-09-26,2.0,0.0,0.056130,0.0,4.0,0.054585,0.055330,1.0,0.340204,2.0,0.0,6.0,0.0
2883,002259d195fcb87b15b98503ac5d2c4d29bc8383053ca3...,2017-10-12,2.0,0.0,0.055060,0.0,4.0,0.056789,0.051328,1.0,0.335022,2.0,0.0,6.0,0.0
2884,002259d195fcb87b15b98503ac5d2c4d29bc8383053ca3...,2017-11-25,2.0,0.0,0.050401,0.0,4.0,0.053355,0.056164,1.0,0.339719,2.0,0.0,6.0,0.0
2885,002259d195fcb87b15b98503ac5d2c4d29bc8383053ca3...,2017-12-24,2.0,0.0,0.063694,0.0,5.0,0.062176,0.056593,1.0,0.347311,2.0,0.0,6.0,0.0
2886,002259d195fcb87b15b98503ac5d2c4d29bc8383053ca3...,2018-01-23,2.0,0.0,0.061535,0.0,5.0,0.061184,0.061361,1.0,0.340421,2.0,0.0,6.0,0.0
2887,002259d195fcb87b15b98503ac5d2c4d29bc8383053ca3...,2018-02-24,2.0,0.0,0.058943,0.0,5.0,0.064558,0.058082,1.0,0.341687,2.0,0.0,6.0,0.0


So my initial hypothesis was incorrect! Although 60922 customers do have rolling `NA` from first observation, however there are 6879 `NA` in the mix which may or may not be missing at random. So as in cluster ONE, we have two types of `NA` here! And things gets really complicated from here. One thing is obvious - `NA` count features for rolling first `NA` could be very useful for the models. However it is not clear if other `NA` should be included in those calculations or not! I am not convinced that they should. But the models could help you answer that!

# Summary

This dataset is rich of `NA` information. We have both systemic `NA` and very likely missing at random `NA`. This means feature-wise `NA` should not be treated equally when doing any kind of feature engineering.

On top of that, I have showed you that it is possible to identify many interesting `NA` properties just by looking at 100 rows in the dataset.

Hope you enjoyed this one. I sure did while I was making this!